In [1]:
#Import libraries

import nilearn as ni
import numpy as np
import matplotlib.pyplot as plt 
from numba import njit,prange
import random

from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets, plotting


In [2]:
file_data1 = np.loadtxt("rsFMRI_0364_1.txt") #Import BOLD temporal series

A = np.array(file_data1)

corrMatrix = np.corrcoef(np.transpose(A)) #Creates functional matrix

np.fill_diagonal(corrMatrix,0)


In [3]:
dt = 0.01 #time step
time = 200 #total time
num_steps = int(time/dt) #Number of steps

nodes = 306 #Number of nodes

omega = 15
alpha = 0.2 #Numerical parameters of our system
lamb = 0.45
beta = 0.05

num_all_sim = 200 #Total number of simulations

In [ ]:
#Function that realizes one simulation
@njit
def brain_simulation(re,im):
    
    z = np.empty((nodes,num_steps))
    
    for i in range(num_steps):
        for k in prange(nodes):

            dx = (lamb-1)*re[k] - (omega*im[k]) + 2*re[k]*(re[k]**2 + im[k]**2) - re[k]*(re[k]**2 + im[k]**2)**2
            dy = (lamb-1)*im[k] + (omega*re[k]) + 2*im[k]*(re[k]**2 + im[k]**2) - im[k]*(re[k]**2 + im[k]**2)**2


            re[k] += dx*dt
            im[k] += dy*dt

            re[k] += beta*dt*np.sum(corrMatrix[:,k]*(re[:]-re[k]))
            im[k] += beta*dt*np.sum(corrMatrix[:,k]*(im[:]-im[k])) 

            re[k] += alpha*np.sqrt(dt)*random.gauss(0,1)
            im[k] += alpha*np.sqrt(dt)*random.gauss(0,1)
            
        z[:,i] = np.sqrt((re)**2 + (im)**2)
            
    return z

In [ ]:
@njit
def all_simulations(W):
    
    for m in prange(num_all_sim):
        
        re = np.random.rand(nodes)*0.05
        im = np.random.rand(nodes)*0.05
        
        W[:,:,m] = brain_simulation(re,im)
    
    return W
